In [35]:


from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON


list_of_prefix = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX prop: <http://dbpedia.org/property/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX res: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX db: <http://dbpedia.org/>
PREFIX yago: <http://dbpedia.org/class/yago/>
PREFIX onto: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX dbpedia2: <http://dbpedia.org/property/>
"""

# will change it to be received from the user

input_query = """
SELECT  ?album WHERE{
   ?album  dbo:rtf dbr:The_Rolling_Stones.
        ?album  rdf:type dbo:Album.
  ?album dbo:recordLabel dbr:London_Records
}
"""
parsed_query = SPARQL(input_query)


# print(parsed_query.triple_list)

class Triple: 
    def __init__(self, sub, pre, obj):
        self.sub = sub
        self.pre = pre
        self.obj = obj

    def get_subject(self):

        return self.sub

    def get_predicate(self):
        return self.pre

    def get_object(self):
        return self.obj



triples_to_query = []

for s in parsed_query.abbr_triple_list:
    triples_to_query.append(
        Triple(s.split(' ')[0], s.split(' ')[1], s.split(' ')[2]))
    

original_query_t = [[],[],[]]

for x in triples_to_query:
    original_query_t[0].append(str(x.get_subject()))
    original_query_t[1].append(str(x.get_predicate()))
    original_query_t[2].append(str(x.get_object()))

# print(len(parsed_query.triple_list))
# parsed_query.abbr_triple_list[1]
# print(parsed_query.abbr_triple_list)
# print(len(parsed_query.abbr_triple_list))
# print(parsed_query.triple_list)
# print(len(parsed_query.triple_list))
# len(triples_to_query)

# for x in triples_to_query:
#     print(x.get_subject(), x.get_predicate(), x.get_object())
#     print("------------------")



## The parser class ##

## The approx function call 

In [39]:


# STARTING POINT APPROX -> GET APPROX -> CALCULATE -> EXECUTE_Q -> BUILD_QUERY -> run_query -> wr_my_query

#  Goes to the dictionary and creates a new index which is the cost and calls the approx queries to calculate the values in that cost 

approx_queries = dict()

def approx(cost):
    for a in range (cost+1):
        approx_queries[str(a)] = get_approx(a)


#  this gets the index(cost) and returns a whole list of triples list for that cost 
# the output is the final list
def get_approx(a):
    value = []
    if a == 0:
        value.append(triples_to_query)
        # build_query(value)
        execute_q(value,a)
    else:
        pre_approx = approx_queries[str(a-1)] # here we get the list of triples list belonging to the previous query
        for triple in pre_approx: # here we iterate over the list of triples list
            new_approx =  calculate(triple, a) # here creating the temp list of triples by calling the calculate function(A single list of triples , cost)
            execute_q(new_approx,a)
            # build_query(new_approx)
            value += new_approx # here we append the list belonging to the previous query to the total list
    return value


# approx(1)



# for key in approx_queries.keys():
#     print (f" ******* key = {key} *******")
#     for b in approx_queries[str(key)]: 
#         # print(len(b))  # b is  a list of triples
#         for c in b: # c is a list triple
#             print((str(c.get_subject()) + " " + str(c.get_predicate()) + " " + str(c.get_object())))
#         print ("-----------------------")
       


## calculation the approx 

In [11]:

def calculate(triples,cost):
    values = []
    for triple in triples:
        index = triples.index(triple)

        if(triple.get_subject() in original_query_t[0]):
            values.append(left_insertion(triples, cost,index))
        if(triple.get_predicate() in original_query_t[1]): 
            values.append(sub(triples, cost, index))
        if(triple.get_object() in original_query_t[2]):
             values.append(right_insertion(triples, cost,index))
    return values
    

def left_insertion(triple,cost,index):
    list_of_approx = triple[:]
    list_of_approx.insert(index,(Triple(list_of_approx[index].get_subject(),f"?p{cost}",f"?o{cost}")))
    list_of_approx[index +1] = Triple(f"?o{cost}",list_of_approx[index+1].get_predicate(),list_of_approx[index+1].get_object())
    return list_of_approx

def right_insertion(triple,cost,index):

    list_of_approx = triple[:]
    list_of_approx.insert(index+1,(Triple(f"?o{cost}",f"?p{cost}",list_of_approx[index].get_object())))
    list_of_approx[index] = Triple(list_of_approx[index].get_subject(),list_of_approx[index].get_predicate(),f"?o{cost}")
    return list_of_approx

def sub(triples,cost,index):
    list_of_approx = triples[:]
    list_of_approx[index] = (Triple(triples[index].get_subject(),f"?p{cost}",triples[index].get_object()))
    return list_of_approx       

## The query builder ##
    

In [77]:

def execute_q(queries,cost):
   for query in queries:
    run_query( build_query(query),cost)
        # build_query(query)


def build_query(queries):
    # label = parsed_query.former.strip(parsed_query.intent)
    sparql_query = f"{parsed_query.intent} distinct{parsed_query.former.strip(parsed_query.intent)} where {{\n"
    for triple in queries:
        sparql_query += f"{triple.get_subject()} {triple.get_predicate()} {triple.get_object()} .\n"
    # sparql_query += f"{label} rdfs:label ?label .\n"
    sparql_query += "}\n LIMIT 25"

#
    return( list_of_prefix + sparql_query)
    # print ( list_of_prefix + sparql_query)

## The endpoint ## 

In [13]:



def run_query(query_to_execute,cost): 
    sparql = SPARQLWrapper("https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
    sparql.setQuery(query_to_execute)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()  
    
    index = results['head']['vars']
    
    ind_reslut = []
    result_list = []
    
    
    for y in index:
        for x1 in results['results']['bindings']:
            ind_reslut.append(x1[y]['value'])
        result_list.append(ind_reslut)
        ind_reslut = []
        
    wr_my_list(result_list,cost)



In [63]:
def wr_my_list(l1list,cost):
    # final_list.append(l1list)
    # final_list.extend( l1list)    
    file_path = 'test2.txt'
    with open(file_path, 'a') as file:
        for l1 in zip(*l1list):
            for l2 in l1:
                final_list.append(l2)
                file.write(str(l2) + " " + str(cost) + "\n" )
   


In [79]:
final_list = []
approx(1)
len(final_list)



18

### To do 

    1- get the labels instead of the uri or have both 
        1.1 for this you first need to get the variables if they are many you need to loop at the end to get them all 
    
    2- Find a way even if a query crashes you keep going on 
        2.1 i think here we need threads 

    3- add the empty path 

    4- continue from where it stopped 
        4.1 basically continue the dictionary 
            4.1.1 problem is it depends on the previous part so we need some sort of cashing to not repeat the whole cost cause for higher costs it impossible. 

        